In [ ]:
pip install parfit

  Created wheel for parfit: filename=parfit-0.220-py3-none-any.whl size=8662 sha256=d836642aae00457ee1858e7b2a5915a390cc3330032172ce7e2ffa847dfd5bce
  Stored in directory: c:\users\mbeke\appdata\local\pip\cache\wheels\9a\3b\19\64addee68f796a7e7108c2658a8765d600753bc728e7beb55e
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=d8b759a133f3d087710879e334fc060fc5e8e4caab3348dcf00d1e1e8b0186c3
  Stored in directory: c:\users\mbeke\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built parfit sklearn
Note: you may need to restart the kernel to use updated packages.


In [91]:
import numpy as np
import pandas as pd
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
import os
import re, string
#import parfit.parfit as pf
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

In [92]:
train_set = pd.read_csv('train_set.csv')
test_set = pd.read_csv('test_set.csv')

In [ ]:
train_set.columns

Index(['lang_id', 'text'], dtype='object')

In [54]:
train_set['lang_id'].value_counts()

nso    3000
eng    3000
ven    3000
afr    3000
xho    3000
tsn    3000
tso    3000
zul    3000
nbl    3000
sot    3000
ssw    3000
Name: lang_id, dtype: int64

In [ ]:
output = re.sub(r'\d+', '', '123hello 456worldtša !')
print(output)

hello worldtša !


In [93]:
def text_cleaner(text):
    
    # Conversion  to lowercase
    text = text.lower()
    # Removal of numbers 
    text = re.sub(r'\d+', '', text)
    # Removal of extra whitespace 
    text = re.sub(r'\s\s+', ' ', text)
    # Removal of space in front/beginning 
    text = text.strip(' ')
    
    #  removal of punctuation
    text = ''.join([ch for ch in text if ch not in string.punctuation])

    
    return text

In [5]:
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [39]:
mzansi = ['eastern','cape', 'free' , 'state','gauteng', 'kwaZulu','natal', 'limpopo', 'mpumalanga', 'north',
          'west', 'northern' , 'western', 'port' 'elizabeth', 'bloemfontein', 'johannesburg', 'durban',
          'polokwane', 'mbombela', 'klerksdorp', 'kimberley', 'town', 'johannesburg', 'pietersburg', 'nelspruit' ]

In [40]:
def remove_mzansi(text):
    return  " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() not in mzansi)

In [6]:
def remove_english(text):
    #words = set(nltk.corpus.words.words())
    return  " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() not in words)

In [7]:
#  testing remove english
remove_english('izolo yesterday namhlanje today')

'izolo namhlanje'

Removing South African Cities and provinces from the data did not improve model performance, neither did removing english words from the non-english languages

In [94]:
train_set['text'] = train_set['text'].apply(text_cleaner)

In [26]:
#for index, row in train_set.iterrows():
#  if row['lang_id'] != 'eng':
#    train_set.iloc[index, 1] = remove_english(train_set.iloc[index, 1])

In [106]:
#train_set['text'] = train_set.loc[train_set['lang_id'] != 'eng']['text'].apply(remove_english)

In [107]:
train_set.iloc[12]

lang_id                                                  zul
text       itipoffs anonymous wusizo locingo oluzimele fu...
Name: 12, dtype: object

In [95]:
X = train_set['text'].values
y = train_set['lang_id'].values

In [96]:
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=.01,
                                                  random_state=42
                                                  )

In [97]:
vectorizer = TfidfVectorizer(analyzer = 'char',
                             ngram_range=(5, 5))

pipe_naive = Pipeline([('vectorizer', vectorizer), ('naive_bayes', MultinomialNB(alpha = 0.5, fit_prior = True))])

In [ ]:
pipe_naive.get_params()

{'memory': None,
 'steps': [('vectorizer',
   TfidfVectorizer(analyzer='char', ngram_range=(5, 5))),
  ('naive_bayes', MultinomialNB())],
 'verbose': False,
 'vectorizer': TfidfVectorizer(analyzer='char', ngram_range=(5, 5)),
 'naive_bayes': MultinomialNB(),
 'vectorizer__analyzer': 'char',
 'vectorizer__binary': False,
 'vectorizer__decode_error': 'strict',
 'vectorizer__dtype': numpy.float64,
 'vectorizer__encoding': 'utf-8',
 'vectorizer__input': 'content',
 'vectorizer__lowercase': True,
 'vectorizer__max_df': 1.0,
 'vectorizer__max_features': None,
 'vectorizer__min_df': 1,
 'vectorizer__ngram_range': (5, 5),
 'vectorizer__norm': 'l2',
 'vectorizer__preprocessor': None,
 'vectorizer__smooth_idf': True,
 'vectorizer__stop_words': None,
 'vectorizer__strip_accents': None,
 'vectorizer__sublinear_tf': False,
 'vectorizer__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vectorizer__tokenizer': None,
 'vectorizer__use_idf': True,
 'vectorizer__vocabulary': None,
 'naive_bayes__alpha': 1.0,
 'na

In [98]:
grid_params = {
  'vectorizer__max_df': np.linspace(0.5, 1.5, 6),
  'vectorizer__binary': [True, False],
}
clf = GridSearchCV(pipe_naive , grid_params)
clf.fit(X_train, y_train)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

Best Score:  0.9998163452708907
Best Params:  {'vectorizer__binary': True, 'vectorizer__max_df': 0.5}


In [85]:
pipe_naive.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', binary=True,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.5, max_features=None,
                                 min_df=1, ngram_range=(5, 5), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('naive_bayes',
                 MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True))],
         verbose=False)

In [99]:
y_pred = clf.predict(X_val)

In [100]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         afr       1.00      1.00      1.00        26
         eng       1.00      1.00      1.00        36
         nbl       1.00      1.00      1.00        25
         nso       1.00      1.00      1.00        26
         sot       1.00      1.00      1.00        31
         ssw       1.00      1.00      1.00        25
         tsn       1.00      1.00      1.00        35
         tso       1.00      1.00      1.00        21
         ven       1.00      1.00      1.00        28
         xho       1.00      1.00      1.00        35
         zul       1.00      1.00      1.00        42

    accuracy                           1.00       330
   macro avg       1.00      1.00      1.00       330
weighted avg       1.00      1.00      1.00       330



In [103]:
test_set['text'] = test_set['text'].apply(text_cleaner)
test_set['lang_id'] = clf.predict(test_set['text'])

In [104]:
test_set.drop('text', axis = 1)

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr
...,...,...
5677,5678,eng
5678,5679,nso
5679,5680,sot
5680,5681,sot


In [105]:
test_set[['index','lang_id']].to_csv('submission25.csv', index=False)

In [ ]:
submission = pd.read_csv('submission.csv')

In [ ]:
submission.head()

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr


In [ ]:
train_set.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...
